this script takes input for many SerialNumbers and gives out their thermal resistance

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path 

In [2]:
def find_coeff(fWavelength):
    if (fWavelength >= 630) and (fWavelength < 808):
        ffnWaveTempCoeff = 0.000751445 * fWavelength - 0.32716763
    elif (fWavelength >= 808) and (fWavelength <= 1600):
        ffnWaveTempCoeff = 0.000419162 * fWavelength - 0.058682635 
    else:
        ffnWaveTempCoeff = -1 

    return ffnWaveTempCoeff

create the Thermal Resistance function

In [3]:
def findingRthermal(data):
    data['WaveLength_nm'] = data['WaveLength'] * 1000000000
    data['WasteHeat_W'] = (data['Current'] * data['Voltage']) - data['Optical']

    y = data['WaveLength_nm']
    x = data['WasteHeat_W']
    slope_intercept = np.polyfit(x,y,1)

    intercept = slope_intercept[1]
    
    firstWL = data['WaveLength_nm'].iat[0]
    #print(firstWL)
    ColdWL_cst = find_coeff(firstWL)
    #print(ColdWL_cst)

    data.loc[:,'intercept'] = intercept
    data.loc[:,'ColdWL_cst'] = ColdWL_cst



    data.loc[:,'Temp_Change_C'] = ( data['WaveLength_nm'] - data['intercept'] ) / data['ColdWL_cst'] 

    #ata
    y1 = data['Temp_Change_C']
    x1 = data['WasteHeat_W']

    slope_intercept_1 = np.polyfit(x1,y1,1)
    slope_intercept_1
    Thermal_resistance = slope_intercept_1[0]

    #print('Thermal Resistance =', round( Thermal_resistance, 2 ) , ' degC / W') 
    return round( Thermal_resistance, 2 )

Import the CSV

In [4]:
data = pd.read_csv (r'C://Users//US02244//python_files//Rthermal//p2.csv')
#data

Split the big data file in smaller pieces by SN

In [5]:
grouped = data.groupby(['SN'])
l_grouped = list(grouped)
groups_list_length = len(l_grouped)
#len(l_grouped)
# now we created groups by SN and those groups are in a list
#a = l_grouped[0][1]
#f = a['SN'].iat[0]
#f

In [6]:
df = pd.DataFrame([])

In [7]:
for i in range(len(l_grouped)):
    a = l_grouped[i][1]['SN'].iat[0]
    b = findingRthermal( l_grouped[i][1] )
    df[i] = [a, b]
    

In [8]:
final_table = df.T
filepath = Path('C://Users//US02244//python_files//Rthermal//file_will_all_Rth_2.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_table.to_csv(filepath) 